In [1]:
import casadi as ca
import numpy as np
from car_model import CarModel
from Milestone1 import *

In [2]:
final_time = 15
car = CarModel()
tf, states, controls, dynamics = car.get_system()

In [3]:
x0 = np.array(
    [
        -30,  # initial x
        1,  # initial y
        10.0,  # initial speed
        0.0,  # beta
        0.0,  # initial psi
        0.0,  # initial wz
        0.0,  # initial delta
    ]
)

In [4]:
w_delta = controls[0]
w_delta_sq = ca.dot(w_delta, w_delta)
quad = tf * w_delta_sq
integrator = ca.integrator('integrator', 'rk', {
    'x': states,
    'p': ca.vertcat(tf, controls),
    'ode': dynamics,
    'quad': quad
})
qf = integrator(x0 = x0, p=ca.vertcat(tf, controls))['qf']
objective = tf + qf
rhs = {'x': states, 'p': ca.vertcat(tf, controls), 'ode': dynamics}

In [5]:
ms = ms_ocp(rhs, nx=7, N=20, T=1, nu=5)

In [6]:
car.get_controls_bounds(1)

(DM([-0.5, 0, 0, 1]), DM([0.5, 15000, 1, 1]))

In [7]:
B = 2
h1, h2, h3, h4 = 1.1*B + 0.25, 3.5, 1.2*B+3.75, 1.3*B + 0.25
setup = add_constraints(
    ms_data      = ms,
    #x_bounds     = car.get_state_bounds(h1, h2, h3, h4),
    u_bounds     = (np.concatenate(([0],car.get_controls_bounds(1)[0].full().squeeze())),
                    np.concatenate(([100],car.get_controls_bounds(1)[1].full().squeeze()))),
    x0_val       = x0,        # fixed initial state
)

In [8]:
solver = ca.nlpsol('solver', 'ipopt', setup['prob'])
sol    = solver(x0=setup['w0'],
                lbx=setup['lbx'], ubx=setup['ubx'],
                lbg=setup['lbg'], ubg=setup['ubg'])

w_opt  = sol['x'].full().flatten()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     1230
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      865

Error evaluating Jacobian of equality constraints at user provided starting point.
  No scaling factors for equality constraints computed!

Number of Iterations....: 0

Number of objective function evaluations             = 0
Number of objective gradient evaluations             = 0
Number of equality constraint evaluations            = 0
Number o

CasADi - 2025-06-16 13:12:31 WARNING("F:daeF failed: NaN detected for output ode, at (row 2, col 0).") [.../casadi/core/oracle_function.cpp:393]
The right-hand side routine failed at the first call.
Function F (0x7fcdbe8d6e00)
Input 0 (x0): [0, 0, 0, 0, 0, 0, 0]
Input 1 (z0): 0x1
Input 2 (p): [0, 0, 0, 0, 1]
Input 3 (u): 0x1
Input 4 (adj_xf): 0x0
Input 5 (adj_zf): 0x0
Input 6 (adj_qf): 0x0
Function nlp_jac_g (0x7fcdbfa3fe00)
Input 0 (x): [-30, 1, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,